In [11]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import math

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (11, 7)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna 

# Feel free to add all the libraries you need
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from geopy.distance import geodesic


In [12]:

# Importing airports data and saving it to a csv file
import airportsdata
airports = airportsdata.load('IATA')

# Filling the missing the airport information for SXF with the data of BER
airports['SXF'] = {'icao': 'EDDB',
 'iata': 'SXF',
 'name': 'Berlin Schonefeld Airport',
 'city': 'Berlin',
 'subd': 'Brandenburg',
 'country': 'DE',
 'elevation': 156.0,
 'lat': 52.362167,
 'lon': 13.500667,
 'tz': 'Europe/Berlin',
 'lid': ''}

airports_df = pd.DataFrame(airports).T
airports_df.to_csv('data/airportdata.csv', index=True)

In [13]:

# Import flight data and airport data
df_flightdata = pd.read_csv('data/Train.csv')
df_airportdata = pd.read_csv('data/airportdata.csv',index_col=0)

# # Converting to the right datatype
df_flightdata['STA'] = pd.to_datetime(df_flightdata['STA'], format='%Y-%m-%d %H.%M.%S')
df_flightdata['STD'] = pd.to_datetime(df_flightdata['STD'], format='%Y-%m-%d %H:%M:%S')
df_flightdata['DATOP'] = pd.to_datetime(df_flightdata['DATOP'], format='%Y-%m-%d')

In [14]:
# Merge df with departure airport data
df = df_flightdata.join(df_airportdata[['iata', 'country', 'elevation', 'lat', 'lon']].add_prefix('dep_'), how='left', on='DEPSTN')
df = df.join(df_airportdata[['iata', 'country', 'elevation', 'lat', 'lon']].add_prefix('arr_'), how='left', on='ARRSTN')
# Merge df with arrival airport data

In [15]:
# FIX COLUMN NAMES Function - general function build out with time
def fix_column_names(input_dataset):
    infun_data = input_dataset
    # pull the starting columns to show at the end the transformation 
    starting_columns = list(infun_data.columns)
    # carry out the changes; remove spaces and hyphens, lower case everything
    infun_data.columns = infun_data.columns.str.replace(' ','_')
    infun_data.columns = infun_data.columns.str.lower()
    infun_data.columns = infun_data.columns.str.replace('-', '_')
    # pull the ending columns to show at the end the transformation 
    ending_columns = list(infun_data.columns)
    # create a dictionary of the old names vs the new names 
    dict_of_names = {'old column name': starting_columns, 'new column name': ending_columns}
    dataframe_of_column_names = df = pd.DataFrame(dict_of_names) 
    print(dataframe_of_column_names)
    return infun_data

df = fix_column_names(df)

   old column name new column name
0               ID              id
1            DATOP           datop
2            FLTID           fltid
3           DEPSTN          depstn
4           ARRSTN          arrstn
5              STD             std
6              STA             sta
7           STATUS          status
8               AC              ac
9           target          target
10        dep_iata        dep_iata
11     dep_country     dep_country
12   dep_elevation   dep_elevation
13         dep_lat         dep_lat
14         dep_lon         dep_lon
15        arr_iata        arr_iata
16     arr_country     arr_country
17   arr_elevation   arr_elevation
18         arr_lat         arr_lat
19         arr_lon         arr_lon


In [16]:
# you must make the dataset equal to the function
# i.e. df = del_duplicates(df)
def del_duplicates(input_dataset):
    # check for duplicates 
    print(input_dataset.duplicated().value_counts())
    # remove duplicates
    input_dataset = input_dataset.drop_duplicates()
    # reset index inplace
    input_dataset.reset_index(inplace=True, drop=True)
    return input_dataset
    
df = del_duplicates(df)

False    107833
Name: count, dtype: int64


In [17]:
# which are statistical continous values?
# which are statistical quantitative values?


# check the different columns
# is a column a unique identifier? 
# can it be dropped to a unique identifier table, 
# i.e. unique identifier and then name, address, ticket number etc. (info table)
# postcode/zipcode an example of a none unique identifier, things that can't be grouped
df.dtypes

def table_nuniques(input_dataset):
    nunique_dictionary = {'column name': [], 'nuniques': [], 'uniques':[], 'dtype' : []}
    infun_data = input_dataset
    columnlist = list(infun_data.columns)
    for i in columnlist: 
        nunique_dictionary['column name'].append(i)
        nunique_dictionary['nuniques'].append(infun_data[i].nunique())        
        nunique_dictionary['uniques'].append(infun_data[i].unique())
        nunique_dictionary['dtype'].append(infun_data[i].dtype)
    # print the dictionary as a dataframe
    return pd.DataFrame(nunique_dictionary)

table_nuniques(df)

,column name,nuniques,uniques,dtype
0,id,107833,"[train_id_0, train_id_1, train_id_2, train_id_...",object
1,datop,1011,"[2016-01-03 00:00:00, 2016-01-13 00:00:00, 201...",datetime64[ns]
2,fltid,1861,"[TU 0712 , TU 0757 , TU 0214 , TU 0480 , TU 03...",object
3,depstn,132,"[CMN, MXP, TUN, DJE, TLS, IST, ORY, MIR, BRU, ...",object
4,arrstn,128,"[TUN, IST, NTE, ALG, BCN, ORY, FCO, NCE, MRS, ...",object
5,std,81697,"[2016-01-03 10:30:00, 2016-01-13 15:05:00, 201...",datetime64[ns]
6,sta,85136,"[2016-01-03 12:55:00, 2016-01-13 16:55:00, 201...",datetime64[ns]
7,status,5,"[ATA, DEP, RTR, SCH, DEL]",object
8,ac,68,"[TU 32AIMN, TU 31BIMO, TU 736IOK, TU 320IMU, T...",object
9,target,968,"[260.0, 20.0, 0.0, 22.0, 53.0, 10.0, 15.0, 16....",float64


In [18]:
#  no na was found!!
print("Null values")
print(df.isnull().sum())
print("="*50)
print("Zero values")
print((df == 0).sum())

Null values
id               0
datop            0
fltid            0
depstn           0
arrstn           0
std              0
sta              0
status           0
ac               0
target           0
dep_iata         0
dep_country      0
dep_elevation    0
dep_lat          0
dep_lon          0
arr_iata         0
arr_country      0
arr_elevation    0
arr_lat          0
arr_lon          0
dtype: int64
Zero values
id                   0
datop                0
fltid                0
depstn               0
arrstn               0
std                  0
sta                  0
status               0
ac                   0
target           38168
dep_iata             0
dep_country          0
dep_elevation        0
dep_lat              0
dep_lon              0
arr_iata             0
arr_country          0
arr_elevation        0
arr_lat              0
arr_lon              0
dtype: int64


In [19]:
# Feature engineering

# Airline names
df['airline_code'] = df['fltid'].str[:2]

# Elevation difference
df['elevation_dif'] = (df['arr_elevation'] - df['dep_elevation'])

# Flight time in minutes
df['flight_time_in_min'] = (df['sta'] - df['std']).dt.total_seconds() / 60

In [20]:
# Distance between departure and arrival airports using the library geodesic
def calculate_distance(row):
    dep_coords = (row['dep_lat'], row['dep_lon'])
    arr_coords = (row['arr_lat'], row['arr_lon'])
    distance = geodesic(dep_coords, arr_coords).kilometers
    return int(round(distance,0))

df['flight_distance_in_km'] = df.apply(calculate_distance, axis=1)

In [21]:
df['average_flight_speed_km_h'] = (df['flight_distance_in_km']*60/df['flight_time_in_min'])
df['average_flight_speed_km_h'] = df['average_flight_speed_km_h'].round().astype(int)


In [22]:
df['international_flight'] = np.where(df['arr_country'] != df['dep_country'], 1, 0)

In [23]:
df['international_flight'].value_counts()

international_flight
1    88932
0    18901
Name: count, dtype: int64

In [24]:
# Extract year, month, and day components
df['year'] = df['datop'].dt.year
df['month'] = df['datop'].dt.month
df['day'] = df['datop'].dt.day

# Create the seasons column
df.loc[(df['month'] < 3) | (df['month'] == 12), 'season'] = 'winter'
df.loc[(df['month'] >= 3) & (df['month'] < 6), 'season'] = 'spring' 
df.loc[(df['month'] >= 6) & (df['month'] < 9), 'season'] = 'summer' 
df.loc[(df['month'] >= 9) & (df['month'] < 12), 'season'] = 'autumn'

In [25]:
df.head().T

,0,1,2,3,4
id,train_id_0,train_id_1,train_id_2,train_id_3,train_id_4
datop,2016-01-03 00:00:00,2016-01-13 00:00:00,2016-01-16 00:00:00,2016-01-17 00:00:00,2016-01-17 00:00:00
fltid,TU 0712,TU 0757,TU 0214,TU 0480,TU 0338
depstn,CMN,MXP,TUN,DJE,TUN
arrstn,TUN,TUN,IST,NTE,ALG
std,2016-01-03 10:30:00,2016-01-13 15:05:00,2016-01-16 04:10:00,2016-01-17 14:10:00,2016-01-17 14:30:00
sta,2016-01-03 12:55:00,2016-01-13 16:55:00,2016-01-16 06:45:00,2016-01-17 17:00:00,2016-01-17 15:50:00
status,ATA,ATA,ATA,ATA,ATA
ac,TU 32AIMN,TU 31BIMO,TU 32AIMN,TU 736IOK,TU 320IMU
target,260.0,20.0,0.0,0.0,22.0


In [26]:
# df['datop']=df['datop'].dt.strftime('%Y%m%d')
# df['std_time']=(df['std'].dt.strftime('%H%M%S'))
# df['sta_time']=(df['sta'].dt.strftime('%H%M%S'))

In [27]:
# Convert timestamps to numeric features
df['std_time'] = df['std'].dt.time
df['sta_time'] = df['sta'].dt.time

In [28]:
df['std_time'] = df['std_time'].astype(str).str.replace(':', '').astype(int)
df['sta_time'] = df['sta_time'].astype(str).str.replace(':', '').astype(int)
df['datop'] = df['datop'].astype(str).str.replace('-', '').astype(int)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107833 entries, 0 to 107832
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   id                         107833 non-null  object        
 1   datop                      107833 non-null  int64         
 2   fltid                      107833 non-null  object        
 3   depstn                     107833 non-null  object        
 4   arrstn                     107833 non-null  object        
 5   std                        107833 non-null  datetime64[ns]
 6   sta                        107833 non-null  datetime64[ns]
 7   status                     107833 non-null  object        
 8   ac                         107833 non-null  object        
 9   target                     107833 non-null  float64       
 10  dep_iata                   107833 non-null  object        
 11  dep_country                107833 non-null  object  

In [30]:
df.to_csv('data/eda_data.csv',index=True)

In [ ]:
# eda_data = df.drop(['id', 'std', 'sta', 'fltid','arr_iata','dep_iata','ac'], axis=1)